In [3]:
import numpy as np
import cv2
import math
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [4]:
radius = 1
numPoints = 8

def get_pixel_else_0(l, idx, idy):
    if idx < int(len(l)) - 1 and idy < len(l[0]):
        return l[idx,idy]
    else:
        return 0

def bilinearInterpolation(x, y, img):
    x1, y1 = int(x), int(y)
    x2, y2 = math.ceil(x), math.ceil(y)

    r1 = (x2 - x) / (x2 - x1) * get_pixel_else_0(img, x1, y1) + (x - x1) / (x2 - x1) * get_pixel_else_0(img, x2, y1)
    r2 = (x2 - x) / (x2 - x1) * get_pixel_else_0(img, x1, y2) + (x - x1) / (x2 - x1) * get_pixel_else_0(img, x2, y2)

    return (y2 - y) / (y2 - y1) * r1 + (y - y1) / (y2 - y1) * r2    

def thresholded(center, pixels):
    out = []
    if len(pixels) > 0:
        out = np.where((pixels-center) >= 0,1,0)
    return out

def getNeighboringPixelsPaperVersion(img,R,P,x,y):
    pixels = []

    indexes = np.array(list(range(0,P)),dtype=np.float)
    dy = -R * np.sin(2 * np.pi * indexes / P)
    dx = R * np.cos(2 * np.pi * indexes / P)

    dy = np.where(abs(dy) < 5.0e-10, 0, dy)
    dx = np.where(abs(dx) < 5.0e-10, 0, dx)

    for point in range(0, P):
        r = y + dy[point]
        c = x + dx[point]

        fr = math.floor(r)
        fc = math.floor(c)

        cr = math.ceil(r)
        cc = math.ceil(c)
        
        rr = np.round(r)
        rc = np.round(c)

        if abs(c-rc) < 10e-7 and abs(r-rr) < 10e-7:
            pixels.append(get_pixel_else_0(img, int(r), int(c)))
        else:
            tr = r - fr
            tc = c - fc

            w1 = (1 - tc) * (1 - tr)
            w2 =      tc  * (1 - tr)
            w3 = (1 - tc) *      tr 
            w4 =      tc  *      tr 

            value = w1*get_pixel_else_0(img, fr, fc) + w2*get_pixel_else_0(img, fr, cc) + \
                    w3*get_pixel_else_0(img, cr, fc) + w4*get_pixel_else_0(img, cr, cc)

            pixels.append(value)

    return pixels


def getNeighboringPixels(img,R,P,x,y):
    pixels = []

    indexes = np.array(list(range(0,P)),dtype=np.float)
    dy = -R * np.sin(2 * np.pi * indexes / P)
    dx = R * np.cos(2 * np.pi * indexes / P)

    dy = np.where(abs(dy) < 5.0e-10, 0, dy)
    dx = np.where(abs(dx) < 5.0e-10, 0, dx)

    for point in range(0, P):
        c = x + dx[point]
        r = y + dy[point]

        if r < 0 or c < 0:
            pixels.append(0)
            continue            
        if int(r) == r:
            if int(c) != c:
                c1 = int(c)
                c2 = math.ceil(c)
                w1 = (c2 - c) / (c2 - c1)
                w2 = (c - c1) / (c2 - c1)
                                
                pixels.append(int((w1 * get_pixel_else_0(img, int(r), int(c)) + \
                               w2 * get_pixel_else_0(img, int(r), math.ceil(c))) / (w1 + w2)))
            else:
                pixels.append(get_pixel_else_0(img, int(r), int(c)))
        elif int(c) == c:
            r1 = int(r)
            r2 = math.ceil(r)
            w1 = (r2 - r) / (r2 - r1)
            w2 = (r - r1) / (r2 - r1)                
            pixels.append((w1 * get_pixel_else_0(img, int(r), int(c)) + \
                           w2 * get_pixel_else_0(img, math.ceil(r), int(c))) / (w1 + w2))
        else:
            pixels.append(bilinearInterpolation(r, c, img))

    return pixels

def genMappingTable(P):
    mapTable = np.zeros(2**P)
    numElms = P+2
    for i in range(0,2**P):
        msb = 128 & i
        lsb = 1 & i
        count = format(i, '#010b').count('01') + format(i, '#010b').count('10') + ((msb >> (P-1)) ^ lsb)

        if count <= 2:
            mapTable[i] = bin(i).count('1')
        else:
            mapTable[i] = P+1

    return mapTable

def LBP(img,R,P):
    lbpImg = img.copy()

    for x in range(0, len(img)):
        for y in range(0, len(img[0])):
            center = img[x,y]
            pixels = getNeighboringPixels(img,R,P,x,y)                    
            values = thresholded(center, pixels)

            res = 0
            for a in range(0, len(values)):
                res += values[a] * (2 ** a)
            lbpImg.itemset((x,y), res)

    return lbpImg


def ULBP(img,R,P):
    uLbpImg = img.copy()
    p = np.array(list(range(1,P)))
    
    for x in range(0, len(img)):
        for y in range(0, len(img[0])):
            center = img[x,y]
            pixels = getNeighboringPixels(img,R,P,x,y)                    
            values = np.array(thresholded(center, pixels))

            res = abs(values[P-1] - values[0])
            res += np.sum(abs(values[p]-values[p-1]))
            uLbpImg.itemset((x,y), res)
                
    return uLbpImg


def LBPriu2(img,P,R):
    uLbpImg = ULBP(img,R,P)
    lbpRiu2Img = img.copy()

    for x in range(0, len(img)):
        for y in range(0, len(img[0])):
            if uLbpImg[x,y] <= 2:
                center = img[x,y]
                pixels = getNeighboringPixels(img,R,P,x,y)                    
                values = np.array(thresholded(center, pixels))
                lbpRiu2Img.itemset((x,y), np.sum(values))
            else:
                lbpRiu2Img.itemset((x,y), P+1)
    return lbpRiu2Img


def calcCLBPMThreshold(img,P,R):
    threshold = 0
    total = 0

    for x in range(0, len(img)):
        for y in range(0, len(img[0])):
            center = img[x,y]
            pixels = getNeighboringPixels(img,R,P,x,y)                    
            sp, mp = LDSMT(center, pixels)
            total += np.mean(mp)

    return total/(len(img)*len(img[0]))

def calcLocalDifferences(img,P,R):
    ld = np.zeros((img.shape[0],img.shape[1],P),dtype=np.float)

    for y in range(0, len(img)):
        for x in range(0, len(img[0])):
            center = img[y,x]
            #pixels = getNeighboringPixels(img,R,P,x,y)              
            pixels = getNeighboringPixelsPaperVersion(img,R,P,x,y)              
            ld[y,x,0:P] = pixels-center

    return ld

def calcTransitions(pattern,P):
    u_value = np.absolute(pattern[:,:,P-1]-pattern[:,:,0])
    u_value += np.sum(np.absolute(pattern[:,:,1:P]-pattern[:,:,0:P-1]),2)
    return u_value

def LDSMT(ld):
    sp = np.where(ld >= 0,1,-1)
    mp = np.absolute(ld)
    return sp, mp

def CLBP_S(sp,P):
    sp = np.where(sp >= 0, 1, 0)
    pp2 = 2**(np.array(list(range(0,P))))
    return np.dot(sp,pp2)

def CLBP_M(mp,P):
    c = np.mean(mp)
    tp = np.where(mp >= c, 1, 0)
    pp2 = (np.array(list(range(0,P))))
    return np.dot(tp,pp2)

def CLBP_C(im):
    c = np.mean(im)
    return np.where(im >= c, 1, 0)

def CLBP_S_riu2(sp,P):
    sp = np.where(sp >= 0, 1, 0)
    #sp_total = np.sum(sp,2)
    #u_value = calcTransitions(sp,P)
    #return np.where(u_value <= 2, sp_total, P+1)
    pp2 = 2**(np.array(list(range(0,P))))
    indexes = np.dot(sp,pp2)    
    return indexes

def CLBP_M_riu2(mp,P):
    c = np.mean(mp)
    tp = np.where(mp >= c, 1, 0)
    # pp2 = np.array([1]*P)
    # tp_total = np.dot(tp,pp2.T)
    # u_value = calcTransitions(tp,P)
    # return np.where(u_value <= 2, tp_total, P+1)
    pp2 = 2**(np.array(list(range(0,P))))
    indexes = np.dot(tp,pp2)    
    return indexes

def genLocalPatterns(img):
    mapping = "lbp"
    dp = calcLocalDifferences(img,numPoints,radius)
    sp, mp = LDSMT(dp)
    lpImg = []
    # "LBP" | "CLBP_S"
    if (mapping == "lbp") or (mapping == "clbp_s"):
        lpImg = CLBP_S(sp,numPoints)
    # "CLBP_S_riu2"
    elif mapping == "clbp_s_riu2":              
        lpImg = CLBP_S_riu2(sp,numPoints)
    # "CLBP_M"
    elif mapping == "clbp_m":               
        lpImg = CLBP_M(mp,numPoints)
    # "CLBP_M_riu2"
    elif mapping == "clbp_m_riu2":               
        lpImg = CLBP_M_riu2(mp,numPoints)
    # "CLBP_M_riu2"
    elif mapping == "clbp_c":               
        lpImg = CLBP_C(mp,numPoints)
    return lpImg

def describeHistogram(img, sp, mp, mapping):
    # "LBP"
    if mapping == "lbp":                    
        lbp = CLBP_S(sp,numPoints)
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 2**numPoints))
    # "CLBP_S_riu2"
    elif mapping == "clbp_s":              
        clbp_s_riu2 = CLBP_S_riu2(sp,numPoints)
        (hist, bins) = np.histogram(clbp_s_riu2.ravel(), bins=np.arange(0, numPoints+3))
    # "CLBP_M_riu2"
    elif mapping == "clbp_m":               
        clbp_m_riu2 = CLBP_M_riu2(mp,numPoints)
        (hist, bins) = np.histogram(clbp_m_riu2.ravel(), bins=np.arange(0, numPoints+3))
    # "CLBP_M_riu2/C"
    elif mapping == "clbp_m/c":             
        clbp_m_riu2 = CLBP_M_riu2(mp,numPoints)
        clbp_c = CLBP_C(img)
        edgeM = list(range(0,numPoints+3))
        edgeC = list(range(0,3))
        hist2d, xedges, yedges = np.histogram2d(clbp_m_riu2.ravel(), 
            clbp_c.ravel(), 
            bins=(edgeM, edgeC))
        hist = hist2d.flatten()
    # "CLBP_S_riu2_M_riu2/C"
    elif mapping == "clbp_s_m/c":           
        clbp_m_riu2 = CLBP_M_riu2(mp,numPoints)
        clbp_c = CLBP_C(img)
        edges = list(range(0,2**P))
        clbp_m_riu2_c, xedges, yedges = np.histogram2d(clbp_m_riu2.flatten(), 
            clbp_c.flatten(), 
            bins=(edges, edges))
        clbp_s_riu2 = CLBP_S_riu2(sp,numPoints)
        hist = np.concatenate(clbp_s_riu2.flatten(),
            clbp_m_riu2_c.flatten(), 
            axis=0)
    # "CLBP_S_riu2/M_riu2"
    elif mapping == "clbp_s/m":             
        clbp_s_riu2 = CLBP_S_riu2(sp,numPoints)
        clbp_m_riu2 = CLBP_M_riu2(mp,numPoints)
        edges = list(range(0,2**P))
        hist2d, xedges, yedges = np.histogram2d(clbp_s_riu2.flatten(), 
            clbp_m_riu2.flatten(), 
            bins=(edges, edges))
        hist = hist2d.flatten()
    # "CLBP_S_riu2/M_riu2/C"
    elif mapping == "clbp_s/m/c":           
        clbp_s_riu2 = CLBP_S_riu2(sp,numPoints)
        feat1 = clbp_s_riu2.flatten()
        feat1.shape = (len(feat1),1)
        clbp_m_riu2 = CLBP_M_riu2(mp,numPoints)
        feat2 = clbp_m_riu2.flatten()
        feat2.shape = (len(feat2),1)
        clbp_c = CLBP_C(img)
        feat3 = clbp_c.flatten()
        feat3.shape = (len(feat3),1)
        feats = np.concatenate((feat1, feat2, feat3), 
            axis=1)
        edges = list(range(0,2**P))
        hist3d, edges = np.histogramdd(r, 
            bins = (edges, edges, edges))
        hist = hist3d.flatten()

    # normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + np.finfo(float).eps)

    return hist



def chiSquared(p,q):
    return np.sum((p-q)**2/(p+q+1e-6))

    

Making X and Y datasets from the image dataset

In [5]:
database = 'F:/CEERI/Outex-TC-00010/000/'
X_data = []
labels = []
# Making X feature vectors
with open(database + "train.txt", "r") as train:
    with open(database + "train_files.txt", "r") as trainFiles:
        train.readline()
        numTrain = int(train.readline().strip())
        print(numTrain)
        i = 0
        for pathFile in trainFiles:
            print(i)
            if i==0:
                pathFile = pathFile[3:]
            img = cv2.imread(pathFile.rstrip())
            X_data.append(img)
            # print(pathFile.strip())
            i+=1
                
# Making Y feature vectors
with open(database + "train.txt", "r") as train:
    with open(database + "train_files.txt", "r") as trainFiles:
        train.readline()
        numTrain = int(train.readline().strip())
        print(numTrain)
        i = 0
        for pathFile in trainFiles:
            if i<numTrain:   
                # print('\n',i)

                # X_data.append(train.readline().split()[0])
                labels.append(train.readline().split()[1])
                # print(labels[i])
                i += 1

480
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

Test-Train-Split


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_data, labels, test_size=0.2, random_state=42)

Training the dataset and adding the classifier to the dataset

In [8]:
radius = 1
numPoints = 8

clbp_s = []
clbp_m = []
clbp_mc = []
clbp_s_mc = []
clbp_sm = []
clbp_smc = []


nn_classifier_clbp_s = make_pipeline(StandardScaler(), SVC(gamma='auto'))
nn_classifier_clbp_m = make_pipeline(StandardScaler(), SVC(gamma='auto'))
nn_classifier_clbp_mc = make_pipeline(StandardScaler(), SVC(gamma='auto'))
nn_classifier_clbp_s_mc = make_pipeline(StandardScaler(), SVC(gamma='auto'))
nn_classifier_clbp_sm = make_pipeline(StandardScaler(), SVC(gamma='auto'))
nn_classifier_clbp_smc = make_pipeline(StandardScaler(), SVC(gamma='auto'))

mappingTable = genMappingTable(numPoints)

# train
i = 0
for imgdata in X_train:
    i+=1
    print(i)

    img_bgr = imgdata
    height, width, channel = img_bgr.shape
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    img = img_gray
    # print("height,width",height,width)
    # cv2.imshow("img",img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # img = img/255
    # img = (img-np.mean(img))/np.std(img)*20+128

    dp = calcLocalDifferences(img,numPoints,radius)
    sp, mp = LDSMT(dp)

    # CLBP_C
    clbp_c = CLBP_C(img)

    # CLBP_S_riu2
    clbp_s_riu2 = CLBP_S_riu2(sp,numPoints)
    mapped_clbp_s_riu2 = mappingTable[clbp_s_riu2]
    (hist_s, bins) = np.histogram(mapped_clbp_s_riu2.ravel(), bins=numPoints+2)
    clbp_s.append(hist_s.ravel())
    
    # CLBP_M_riu2
    clbp_m_riu2 = CLBP_M_riu2(mp,numPoints)
    mapped_clbp_m_riu2 = mappingTable[clbp_m_riu2]
    (hist_m, bins) = np.histogram(mapped_clbp_m_riu2.ravel(), bins=numPoints+2)
    clbp_m.append(hist_m.ravel())

    # CLBP_M/C
    hist_mc, xedges, yedges = np.histogram2d(mapped_clbp_m_riu2.ravel(), clbp_c.ravel(),bins=[numPoints+2,2])
    clbp_mc.append(hist_mc.ravel())

    # CLBP_S_M/C
    hist_s_mc = np.concatenate((hist_s,hist_mc.ravel()),axis=0)
    clbp_s_mc.append(hist_s_mc.ravel())

    # CLBP_S/M
    hist_sm, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(),mapped_clbp_m_riu2.ravel(),bins=[numPoints+2,numPoints+2])
    clbp_sm.append(hist_sm.ravel())

    # CLBP_S/M/C
    clbp_mc_sum = np.where(clbp_c > 0, clbp_m_riu2+numPoints+2, clbp_m_riu2)
    hist_smc, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(), clbp_mc_sum.ravel(),bins=[numPoints+2,2*(numPoints+2)])
    clbp_smc.append(hist_smc.ravel())

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


Fitting the dataset with the classifier

In [9]:
# fitting
print("Fitting the models")

# model = KNeighborsClassifier(n_neighbors = 1, weights='uniform', algorithm='auto', metric=chiSquared)
# model.fit(data,y_train)

nn_classifier_clbp_s.fit(clbp_s,y_train)
nn_classifier_clbp_m.fit(clbp_m,y_train)
nn_classifier_clbp_mc.fit(clbp_mc,y_train)
nn_classifier_clbp_s_mc.fit(clbp_s_mc,y_train)
nn_classifier_clbp_sm.fit(clbp_sm,y_train)
nn_classifier_clbp_smc.fit(clbp_smc,y_train)

Fitting the models


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

Testing the dataset

In [11]:
result_labels_clbp_s = []
result_labels_clbp_m = []
result_labels_clbp_mc = []
result_labels_clbp_s_mc = []
result_labels_clbp_sm = []
result_labels_clbp_smc = []

# test
print("Test steps:")
i = 0
for imgdata in X_test:
    print(i)
    # image_file = '/content/00000'+str(i)+'.bmp'
    img_bgr = imgdata
    height, width, channel = img_bgr.shape
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    img = img_gray
    dp = calcLocalDifferences(img,numPoints,radius)
    sp, mp = LDSMT(dp)

    # CLBP_C
    clbp_c = CLBP_C(img)

    # CLBP_S_riu2
    clbp_s_riu2 = CLBP_S_riu2(sp,numPoints)
    mapped_clbp_s_riu2 = mappingTable[clbp_s_riu2]
    (hist_s, bins) = np.histogram(mapped_clbp_s_riu2.ravel(), bins=numPoints+2)
    result_labels_clbp_s.append((y_test[i],
        nn_classifier_clbp_s.predict([hist_s.ravel()])))
    
    # CLBP_M_riu2
    clbp_m_riu2 = CLBP_M_riu2(mp,numPoints)
    mapped_clbp_m_riu2 = mappingTable[clbp_m_riu2]
    (hist_m, bins) = np.histogram(mapped_clbp_m_riu2.ravel(), bins=numPoints+2)
    result_labels_clbp_m.append((y_test[i],
        nn_classifier_clbp_m.predict([hist_m.ravel()])))


    # CLBP_M/C
    hist_mc, xedges, yedges = np.histogram2d(mapped_clbp_m_riu2.ravel(), clbp_c.ravel(),bins=[numPoints+2,2])
    result_labels_clbp_mc.append((y_test[i],
        nn_classifier_clbp_mc.predict([hist_mc.ravel()])))

    # CLBP_S_M/C
    hist_s_mc = np.concatenate((hist_s,hist_mc.ravel()),axis=0)
    result_labels_clbp_s_mc.append((y_test[i],
        nn_classifier_clbp_s_mc.predict([hist_s_mc.ravel()])))

    # CLBP_S/M
    hist_sm, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(),mapped_clbp_m_riu2.ravel(),bins=[numPoints+2,numPoints+2])
    result_labels_clbp_sm.append((y_test[i],
        nn_classifier_clbp_sm.predict([hist_sm.ravel()])))

    # CLBP_S/M/C
    clbp_mc_sum = np.where(clbp_c > 0, clbp_m_riu2+numPoints+2, clbp_m_riu2)
    hist_smc, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(), clbp_mc_sum.ravel(),bins=[numPoints+2,2*(numPoints+2)])
    result_labels_clbp_smc.append((y_test[i],
        nn_classifier_clbp_smc.predict([hist_smc.ravel()])))
    
    i+=1

Test steps:
0
1
2
3
4


CLBP-S_riu2 predictions results

In [ ]:
pred = 0
numTest = np.shape(X_test)[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_s[j][0] == result_labels_clbp_s[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 88
accuracy = 91.66666666666666


CLBP-M_riu2 predictions results

In [ ]:
pred = 0
numTest = np.shape(X_test)[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_m[j][0] == result_labels_clbp_m[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 91
accuracy = 94.79166666666666


CLBP-M/C predictions results

In [ ]:
pred = 0
numTest = np.shape(X_test)[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_mc[j][0] == result_labels_clbp_mc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 89
accuracy = 92.70833333333334


CLBP-S_M/C predictions results

In [ ]:
pred = 0
numTest = np.shape(X_test)[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_s_mc[j][0] == result_labels_clbp_s_mc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 95
accuracy = 98.95833333333334


CLBP-S/M predictions results

In [ ]:
pred = 0
numTest = np.shape(X_test)[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_sm[j][0] == result_labels_clbp_smc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 96
accuracy = 100.0


CLBP-S/M/C predictions results

In [ ]:
pred = 0
numTest = np.shape(X_test)[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_smc[j][0] == result_labels_clbp_smc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 96
accuracy = 100.0
